# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 14 2022 2:49PM,246451,19,MSP213965,"Methapharm, Inc.",Released
1,Sep 14 2022 2:47PM,246449,10,MSP213970,"Methapharm, Inc.",Released
2,Sep 14 2022 2:47PM,246449,10,MSP213966,"Methapharm, Inc.",Released
3,Sep 14 2022 2:47PM,246450,10,8526208,Yusen – 3D Matrix,Released
4,Sep 14 2022 2:47PM,246448,10,MSP213964,"Methapharm, Inc.",Released
5,Sep 14 2022 2:47PM,246448,10,MSP213967,"Methapharm, Inc.",Released
6,Sep 14 2022 2:47PM,246448,10,MSP213968,"Methapharm, Inc.",Released
7,Sep 14 2022 2:43PM,246447,20,ALU0007930,Alumier Labs Inc.,Released
8,Sep 14 2022 2:43PM,246447,20,ALUR809831162,Alumier Labs Inc.,Released
9,Sep 14 2022 2:43PM,246447,20,ALUR842988174,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,246447,Released,3
42,246448,Released,3
43,246449,Released,2
44,246450,Released,1
45,246451,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246447,NaN,NaN,3.0
246448,NaN,NaN,3.0
246449,NaN,NaN,2.0
246450,NaN,NaN,1.0
246451,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246315,0.0,0.0,1.0
246335,0.0,0.0,1.0
246353,0.0,0.0,2.0
246354,0.0,0.0,1.0
246355,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246315,0,0,1
246335,0,0,1
246353,0,0,2
246354,0,0,1
246355,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246315,0,0,1
1,246335,0,0,1
2,246353,0,0,2
3,246354,0,0,1
4,246355,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246315,,,1
1,246335,,,1
2,246353,,,2
3,246354,,,1
4,246355,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 14 2022 2:49PM,246451,19,"Methapharm, Inc."
1,Sep 14 2022 2:47PM,246449,10,"Methapharm, Inc."
3,Sep 14 2022 2:47PM,246450,10,Yusen – 3D Matrix
4,Sep 14 2022 2:47PM,246448,10,"Methapharm, Inc."
7,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.
10,Sep 14 2022 2:42PM,246446,10,"Evolution Protection, LLC"
11,Sep 14 2022 2:41PM,246444,19,ACG North America LLC
13,Sep 14 2022 1:56PM,246442,19,"GUSA Granules USA, Inc."
14,Sep 14 2022 1:43PM,246441,19,"GUSA Granules USA, Inc."
15,Sep 14 2022 1:39PM,246440,20,"Adare Pharmaceuticals, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 14 2022 2:49PM,246451,19,"Methapharm, Inc.",,,1
1,Sep 14 2022 2:47PM,246449,10,"Methapharm, Inc.",,,2
2,Sep 14 2022 2:47PM,246450,10,Yusen – 3D Matrix,,,1
3,Sep 14 2022 2:47PM,246448,10,"Methapharm, Inc.",,,3
4,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,,,3
5,Sep 14 2022 2:42PM,246446,10,"Evolution Protection, LLC",,,1
6,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,,,2
7,Sep 14 2022 1:56PM,246442,19,"GUSA Granules USA, Inc.",,,1
8,Sep 14 2022 1:43PM,246441,19,"GUSA Granules USA, Inc.",,,1
9,Sep 14 2022 1:39PM,246440,20,"Adare Pharmaceuticals, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 2:49PM,246451,19,"Methapharm, Inc.",1,,
1,Sep 14 2022 2:47PM,246449,10,"Methapharm, Inc.",2,,
2,Sep 14 2022 2:47PM,246450,10,Yusen – 3D Matrix,1,,
3,Sep 14 2022 2:47PM,246448,10,"Methapharm, Inc.",3,,
4,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3,,
5,Sep 14 2022 2:42PM,246446,10,"Evolution Protection, LLC",1,,
6,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,2,,
7,Sep 14 2022 1:56PM,246442,19,"GUSA Granules USA, Inc.",1,,
8,Sep 14 2022 1:43PM,246441,19,"GUSA Granules USA, Inc.",1,,
9,Sep 14 2022 1:39PM,246440,20,"Adare Pharmaceuticals, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 2:49PM,246451,19,"Methapharm, Inc.",1,,
1,Sep 14 2022 2:47PM,246449,10,"Methapharm, Inc.",2,,
2,Sep 14 2022 2:47PM,246450,10,Yusen – 3D Matrix,1,,
3,Sep 14 2022 2:47PM,246448,10,"Methapharm, Inc.",3,,
4,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 2:49PM,246451,19,"Methapharm, Inc.",1.0,NaN,NaN
1,Sep 14 2022 2:47PM,246449,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Sep 14 2022 2:47PM,246450,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Sep 14 2022 2:47PM,246448,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 2:49PM,246451,19,"Methapharm, Inc.",1.0,0.0,0.0
1,Sep 14 2022 2:47PM,246449,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Sep 14 2022 2:47PM,246450,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Sep 14 2022 2:47PM,246448,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4681880,40.0,5.0,0.0
102,985422,4.0,0.0,0.0
15,246384,1.0,1.0,35.0
17,492723,2.0,0.0,0.0
19,1971473,21.0,23.0,5.0
20,985742,18.0,16.0,0.0
21,246315,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4681880,40.0,5.0,0.0
1,102,985422,4.0,0.0,0.0
2,15,246384,1.0,1.0,35.0
3,17,492723,2.0,0.0,0.0
4,19,1971473,21.0,23.0,5.0
5,20,985742,18.0,16.0,0.0
6,21,246315,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40.0,5.0,0.0
1,102,4.0,0.0,0.0
2,15,1.0,1.0,35.0
3,17,2.0,0.0,0.0
4,19,21.0,23.0,5.0
5,20,18.0,16.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,102,Released,4.0
2,15,Released,1.0
3,17,Released,2.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,17,19,20,21
Status,,,,,,,
Completed,0.0,0.0,35.0,0.0,5.0,0.0,0.0
Executing,5.0,0.0,1.0,0.0,23.0,16.0,0.0
Released,40.0,4.0,1.0,2.0,21.0,18.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,17,19,20,21
0,Completed,0.0,0.0,35.0,0.0,5.0,0.0,0.0
1,Executing,5.0,0.0,1.0,0.0,23.0,16.0,0.0
2,Released,40.0,4.0,1.0,2.0,21.0,18.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,17,19,20,21
0,Completed,0.0,0.0,35.0,0.0,5.0,0.0,0.0
1,Executing,5.0,0.0,1.0,0.0,23.0,16.0,0.0
2,Released,40.0,4.0,1.0,2.0,21.0,18.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()